            TODO 
-> save models to further usage
-> player

In [1]:
import pysrt
import joblib


model = joblib.load('C:\dev\whisper_models\whisper_small.pkl')

In [15]:
import os
import srt
from datetime import timedelta

In [39]:
import whisper
import pysrt

model = whisper.load_model("medium")

In [3]:
result = model.transcribe("C:\Movies\My Hero Academia S01 (2016) MULTi 1080p 10bits BluRay x265 AAC v2 -Punisher694\My Hero Academia S01E02 MULTi 1080p 10bits BluRay x265 AAC -Punisher694.mkv")

In [6]:
def convert_time(seconds):

    hours = int(seconds // 3600)
    remaining_seconds = seconds % 3600
    
    minutes = int(remaining_seconds // 60)
    remaining_seconds %= 60
    
    seconds = int(remaining_seconds)
    miliseconds = int((remaining_seconds - seconds) * 1000)
    
    return (hours, minutes, seconds, miliseconds)


In [2]:
def result_to_srt_by_pysrt(result, output_file):
    subs = pysrt.SubRipFile()
    index = 1
    for segment in result['segments']:
        start_time = convert_time(segment['start']) 
        end_time = convert_time(segment['end'])
        
        sub_item = pysrt.SubRipItem(index=index,
                                    start=start_time, 
                                    end=end_time, 
                                    text=segment['text'])
        subs.append(sub_item)
        index += 1
    subs.save(output_file, encoding='utf-8')
    print(f'Subtitle file {output_file} is ready.')   

In [29]:
def result_to_srt_by_srt(result, output_file):
    subs = []
    index = 1
    for segment in result['segments']:
        sub_item = srt.Subtitle(index=index, 
                                start=timedelta(seconds=segment['start']),
                                end=timedelta(seconds=segment['end']), 
                                content=segment['text'])
        index += 1
        subs.append(sub_item)
    subs = srt.compose(subs)
 
    with open(output_file, "w") as f:
        f.write(subs)
        print(f'Subtitle file {output_file} is ready.')   

In [30]:
result_to_srt_by_srt(result, 'output4.srt')

Subtitle file output4.srt is ready.


In [8]:
result_to_srt_by_pysrt(result, 'output3.srt')

Subtitle file output3.srt is ready.


In [3]:
def subtitles_for_folder(video_folder, sub_folder, sub_extension='.srt', srt_function=result_to_srt_by_pysrt):
    files = os.listdir(video_folder)
    file_count = len(files)
    done = 0
    for file in files:
        video_path = os.path.join(video_folder, file)
        result = model.transcribe(video_path)
        
        sub_file = os.path.splitext(video_path)[0] + sub_extension
        sub_path = os.path.join(sub_folder, sub_file)
        if sub_extension == '.srt':
            srt_function(result, sub_path)
            done += 1
            print(f"{done}/{file_count}")
        else:
            #TODO implement other subtitle formats
            print(f"Unable to create {sub_extension} files. Use '.srt' instead.")
            break
        

In [41]:
subtitles_for_folder('C:\\Users\\vlns\\Desktop\\sub_ex2', 'C:\\Users\\vlns\\Desktop\\sub_ex2')

KeyboardInterrupt: 